In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29546
2,Huelva,Confirmados PDIA 14 días,1095
3,Huelva,Tasa PDIA 14 días,"213,3795818149931"
4,Huelva,Confirmados PDIA 7 días,573
5,Huelva,Total Confirmados,29722
6,Huelva,Curados,26500
7,Huelva,Fallecidos,371


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29546.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7722.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 846 personas en los últimos 7 días 

Un positivo PCR cada 426 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29546.0,573.0,1095.0,513170.0,111.658904,213.379582,45.0
Condado-Campiña,9444.0,254.0,491.0,156231.0,162.579770,314.278216,22.0
Huelva-Costa,17142.0,275.0,524.0,289548.0,94.975617,180.971721,19.0
Huelva (capital),7722.0,170.0,338.0,143837.0,118.189339,234.988216,14.0
Sierra de Huelva-Andévalo Central,2619.0,44.0,77.0,67391.0,65.290617,114.258581,4.0
Lucena del Puerto,240.0,7.0,9.0,3261.0,214.658080,275.988960,4.0
Palma del Condado (La),1066.0,21.0,30.0,10801.0,194.426442,277.752060,4.0
Valverde del Camino,501.0,20.0,28.0,12750.0,156.862745,219.607843,3.0
Bollullos Par del Condado,896.0,40.0,88.0,14387.0,278.028776,611.663307,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,39.0,7.0,12.0,1382.0,506.512301,868.306802,0.0
Almonte,1655.0,71.0,158.0,24507.0,289.713143,644.713755,2.0
Bollullos Par del Condado,896.0,40.0,88.0,14387.0,278.028776,611.663307,3.0
Escacena del Campo,154.0,7.0,10.0,2287.0,306.077831,437.254045,1.0
Condado-Campiña,9444.0,254.0,491.0,156231.0,162.579770,314.278216,22.0
Nerva,169.0,7.0,16.0,5169.0,135.422712,309.537628,0.0
Rociana del Condado,534.0,9.0,24.0,7939.0,113.364404,302.305076,1.0
Niebla,205.0,9.0,12.0,4116.0,218.658892,291.545190,0.0
Punta Umbría,754.0,36.0,44.0,15355.0,234.451319,286.551612,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Beas,203.0,1.0,5.0,4341.0,23.036167,115.180834,0.0,0.200000
Aracena,514.0,1.0,5.0,8255.0,12.113870,60.569352,0.0,0.200000
Cartaya,1292.0,7.0,27.0,20083.0,34.855350,134.442065,0.0,0.259259
Palos de la Frontera,605.0,8.0,27.0,11742.0,68.131494,229.943792,2.0,0.296296
Ayamonte,1297.0,5.0,16.0,21104.0,23.692191,75.815011,1.0,0.312500
Rociana del Condado,534.0,9.0,24.0,7939.0,113.364404,302.305076,1.0,0.375000
Bonares,260.0,3.0,7.0,6060.0,49.504950,115.511551,1.0,0.428571
Nerva,169.0,7.0,16.0,5169.0,135.422712,309.537628,0.0,0.437500
Almonte,1655.0,71.0,158.0,24507.0,289.713143,644.713755,2.0,0.449367
